<a href="https://colab.research.google.com/github/ssimpore/IBM-R/blob/master/Indicateur.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

In [2600]:
# Maintriser Pandas
import pandas as pd
from datetime import datetime
import matplotlib.pyplot as plt

In [2601]:
# Lire la base de donnée
from google.colab import files
import io
#uploaded=files.upload()

In [2602]:
data=pd.read_excel('/content/exportAcces_2020_09_30.xls')
data.rename(columns={'Numéro':'numero','Parc':'parc','Type Maintenance':'maintenance','Technologie':'techno','Ouvrage':'ouvrage','Etat':'etat','Valideur':'valideur','Entreprise':'entreprise','CA':'charge_acces',
                   "Date d'autorisation":'auto',"Date début":'debut',"Date de fin":'fin','Commentaire début':'comment_debut','Commentaire fin':'comment_fin','Interventions associées':'intervention','Arrêts associés':'arret'},inplace=True)
data=data.drop(columns=['numero'])
choix=['parc','techno','etat','valideur','debut','auto','fin','comment_debut']
data=data[choix]

In [2603]:
def select_by_date(data_frame,start_end_date):
  start_end_date
  data_temp=data_frame.copy()
  print(len(data_temp))
  test=data_temp['debut'].apply(lambda x:(x>=start_end_date[0])&(x<=start_end_date[1]))
  data_temp=data_temp[test]
  print(len(data_temp))

  return data_temp

In [2604]:
def start_end(data_frame):
  return data_frame['debut'].min(),data_frame['debut'].max()

In [2605]:
def set_hour_bound(x,borne):
  hour=str(x)[11:]
  date=str(x)[:10]
  trop_tot=hour<str(borne[0])
  trop_tard=hour>str(borne[1])
  if trop_tot:
    new_hour=str(borne[0])
    new_date_hour=date+' '+new_hour
  elif trop_tard:
    new_hour=str(borne[1])
    new_date_hour=date+' '+new_hour
  else: new_date_hour=str(x)
  return new_date_hour

In [2606]:
def Filtrage(data_frame, technologie='', etat_acces='', type_acces=[], list_valideur=[]):
    df_temp = data_frame.copy()
    print(f"==> Nombre d'accès avant filtrage : {len(df_temp)}")

    if technologie != '':
        df_temp = df_temp[df_temp['techno'] == technologie]
    else:
        pass

    if etat_acces != '':
        df_temp = df_temp[df_temp['etat'] == etat_acces]
    else:
        pass
        
    if (type_acces != []) and (type_acces != ['']):
        type_acces = list(map(lambda x: str(x).lower(), type_acces))
        df_temp = df_temp[df_temp['comment_debut'].apply(lambda x: str(x)[:2].lower()).isin(type_acces)]
    elif type_acces == ['']:
        df_temp = df_temp[df_temp['comment_debut'].isna()]
    elif type_acces == []:
        pass
    if list_valideur != []:
        list_valideur = list(map(lambda x: str(x).lower(), list_valideur))
        df_temp = df_temp[df_temp['valideur'].apply(lambda x: str(x).lower()).isin(list_valideur)]
    elif list_valideur == []:
        pass
    print(f"==> Nombre d'accès après filtrage : {len(df_temp)}")
    df_temp=df_temp.reset_index()
    
    return df_temp

In [2607]:
def applied_start_end(data_frame,borne):
  data_temp=data_frame.copy()
  data_temp['debut']=data_temp['debut'].apply(lambda x:set_hour_bound(x,borne))
  data_temp['fin']=data_temp['fin'].apply(lambda x:set_hour_bound(x,borne))
  return data_temp

In [2608]:
def diff_two_date(x,y):
  dformat="%d/%m/%Y %H:%M"
  return datetime.strptime(str(x),dformat)-datetime.strptime(str(y),dformat)

In [2609]:
def indicator_calculation(data_frame):
  data_temp=data_frame.copy()
  data_temp['react']=data_temp.apply(lambda x:diff_two_date(x['auto'],x['debut']),axis=1)
  data_temp['arret']=data_temp.apply(lambda x:diff_two_date(x['fin'],x['debut']),axis=1)
  return data_temp

In [2610]:
periode_etude_date_heure=('30/09/2020 00:00','30/09/2020 23:59')
horaire_travail=('06:00','19:00')

techno='Solaire'
etat='terminé'
acces=[]
valideur=[]

df=select_by_date(data,periode_etude_date_heure)
df=Filtrage(df,techno,etat,acces,valideur)
df=applied_start_end(df,horaire_travail)


164
164
==> Nombre d'accès avant filtrage : 164
==> Nombre d'accès après filtrage : 48


In [2611]:
df=indicator_calculation(df)
df[['parc','valideur','debut','fin','auto','react','arret']].head(10)

,parc,valideur,debut,fin,auto,react,arret
0,Arsac 1,SEYE,30/09/2020 17:58,30/09/2020 18:25,30/09/2020 18:12,0 days 00:14:00,0 days 00:27:00
1,Arsac 1,SEYE,30/09/2020 17:08,30/09/2020 17:30,30/09/2020 17:20,0 days 00:12:00,0 days 00:22:00
2,Soleol,SEYE,30/09/2020 17:00,30/09/2020 17:30,30/09/2020 17:02,0 days 00:02:00,0 days 00:30:00
3,Blond,NSIMBA,30/09/2020 16:00,30/09/2020 16:10,30/09/2020 16:01,0 days 00:01:00,0 days 00:10:00
4,Walon Rivesaltes C,NSIMBA,30/09/2020 08:40,30/09/2020 16:00,30/09/2020 15:59,0 days 07:19:00,0 days 07:20:00
5,Walon Rivesaltes A,NSIMBA,30/09/2020 08:40,30/09/2020 16:00,30/09/2020 15:59,0 days 07:19:00,0 days 07:20:00
6,Tiper 1,NSIMBA,30/09/2020 15:10,30/09/2020 15:50,30/09/2020 15:17,0 days 00:07:00,0 days 00:40:00
7,Drambon,NSIMBA,30/09/2020 14:28,30/09/2020 15:15,30/09/2020 14:46,0 days 00:18:00,0 days 00:47:00
8,Arsac 2,NSIMBA,30/09/2020 14:40,30/09/2020 15:00,30/09/2020 14:27,-1 days +23:47:00,0 days 00:20:00
9,Charleval,NSIMBA,30/09/2020 08:00,30/09/2020 13:40,30/09/2020 13:45,0 days 05:45:00,0 days 05:40:00
